In [29]:
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns
%config InlineBackend.figure_format = 'retina'
from pylab import rcParams
rcParams['figure.figsize'] = (9, 6)
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import tqdm
import seaborn as sns
%pylab inline
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

#устраним ошибки со шрифтами
from matplotlib import rcParams
rcParams['font.family'] = 'sans-serif'
rcParams['font.sans-serif'] = ['DejaVu Sans']
import pandas as pd
import numpy as np
from sklearn import metrics
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import *
from sklearn.metrics import *
import random
import warnings
warnings.filterwarnings('ignore')
from sklearn import preprocessing

Populating the interactive namespace from numpy and matplotlib


In [30]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [31]:
test = pd.read_csv('test.csv')

In [32]:
#a = train[[ 'MSSubClass', 'LotFrontage', 'LotArea',  'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd','MasVnrArea', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1', 'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating','HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual', 'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType', 'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual','WoodDeckSF', 'OpenPorchSF','EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'PoolQC','Fence', 'MiscFeature', 'MiscVal', 'MoSold', 'YrSold', 'SalePrice']]
#print(a.corr())


In [33]:
y_train = train['SalePrice']

In [34]:
a = []
for columns in train:
    a += [train[columns].corr(train['SalePrice'], method= 'spearman')]
b = ["{:15} {:.5f}".format(x, y) for y, x in zip(a,train.columns)]

In [35]:
x = pd.DataFrame(a, train.columns).reset_index()
x.columns = ['feature', 'corr']

feature = []
for i in range(81):
    if abs(x['corr'][i]) < 0.15:
        feature += [x.feature[i]]
feature.extend(['Alley', 'PoolQC', 'Fence'])

In [36]:
feature = []
for i in range(81):
    if abs(x['corr'][i]) < 0.15:
        feature += [x.feature[i]]

In [37]:
train.drop(feature, axis=1, inplace=True)

In [38]:
train.shape

(1460, 49)

In [39]:
y = train['SalePrice']

In [40]:
del train['SalePrice']

In [41]:
test.drop(feature, axis=1, inplace=True)

In [42]:
test.shape

(1459, 48)

In [43]:
train_dummies = pd.get_dummies(train)

In [44]:
train_dummies.shape

(1460, 156)

In [45]:
test_dummies = pd.get_dummies(test)

In [46]:
test_dummies.shape

(1459, 152)

In [47]:
dummy_cols = list(set(train_dummies) & set(test_dummies))
dummy_train = train_dummies[dummy_cols]
dummy_test = test_dummies[dummy_cols]

X_train = dummy_train.fillna(-999)
X_test = dummy_test.fillna(-999)

In [48]:
from sklearn.preprocessing import Imputer
imputer = Imputer(missing_values='NaN', strategy='mean', axis=0, verbose=0, copy=True)
imputer.fit(dummy_train)
X_train_imputed = imputer.transform(dummy_train)
X_train_imputed = pd.DataFrame(X_train_imputed, columns=dummy_train.columns.values.tolist())

In [49]:
imputer.fit(dummy_test)
X_test_imputed = imputer.transform(dummy_test)
X_test_imputed = pd.DataFrame(X_train_imputed, columns=dummy_test.columns.values.tolist())

In [50]:
X_train, X_test, y_train, Y_test = train_test_split( X_train_imputed, y, test_size = 0.3, random_state = 0 )

In [51]:
from sklearn.model_selection import cross_val_score, StratifiedKFold, GridSearchCV
from sklearn.metrics import accuracy_score

In [52]:
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=1000)

results = cross_val_score(clf_rf, X_train, y_train, cv=skf)
print("CV accuracy score: {:.2f}%".format(results.mean()*100))

In [53]:
X_train.shape

(1022, 152)

In [54]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(max_depth=1000, max_features=144, max_leaf_nodes=10)
clf.fit(X_train, y_train)

#plt.barh(np.arange(len(clf.feature_importances_)), clf.feature_importances_)
#plt.yticks(np.arange(len(X_train.columns)),X_train.columns)
#''

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=1000,
            max_features=144, max_leaf_nodes=10, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')

In [55]:
#results = cross_val_score(clf, X_train, y_train, cv=skf)
#print("CV accuracy score: {:.2f}%".format(results.mean()*100))

from sklearn.metrics import mean_squared_error
from math import sqrt

y_pred_train = clf.predict_proba(X_train)[:, 1]
rms = sqrt(mean_squared_error(y_train, y_pred_train))

In [56]:
def rmse(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2).mean())

rmse_val = rmse(y_pred_train, y_train)
print("rms error is: " + str(rmse_val))

rms error is: 196751.47285351405


In [58]:
from sklearn.ensemble import RandomForestClassifier
clf_rf = RandomForestClassifier(n_estimators=1000, max_depth=1000, min_samples_leaf=1,max_leaf_nodes=10 , max_features=144, n_jobs=-1)
clf_rf.fit(X_train, y_train)  

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=1000, max_features=144, max_leaf_nodes=10,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=-1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [59]:
results_rand_for = cross_val_score(clf_rf, X_train, y_train, cv=skf)
print("CV accuracy score: {:.2f}%".format(results_rand_for.mean()*100))

CV accuracy score: 10.36%


In [116]:
from sklearn.ensemble import BaggingClassifier

bag_clf = BaggingClassifier(n_estimators=1000, base_estimator=clf, n_jobs=-1)
bag_clf.fit(X_train, y_train)
results_bag_clf = cross_val_score(bag_clf, X_train, y_train, cv=skf)
print("CV accuracy score: {:.2f}%".format(results_bag_clf.mean()*100))


CV accuracy score: 10.68%


In [67]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor

In [113]:
rfr = RandomForestRegressor(max_depth=100)
rfr.fit(X_train, y_train)
#rfr.score(X_test, Y_test)
results_rfr = cross_val_score(rfr, X_train, y_train, cv=skf)
print("CV accuracy score: {:.2f}%".format(results_rfr.mean()*100))

CV accuracy score: 44.83%


In [116]:
y_pred_rfr = rfr.predict(X_test)
rms = sqrt(mean_squared_error(Y_test, y_pred_rfr))
rms

31609.025465904168

In [101]:
knn1 = KNeighborsRegressor(n_neighbors=3)
knn1.fit(X_train, y_train)
results_knn1 = cross_val_score(knn1, X_train, y_train, cv=skf)
print("CV accuracy score: {:.2f}%".format(results_knn1.mean()*100))

CV accuracy score: -1939.13%


In [39]:
y_pred_rf_train_knn1 = knn1.predict(X_train)
rms = sqrt(mean_squared_error(y_train, y_pred_rf_train_knn1))
rms

33048.9482671835

In [75]:
knn2 = KNeighborsRegressor(n_neighbors=300)
knn2.fit(X_train, y_train)
results_knn2 = cross_val_score(knn2, X_train, y_train, cv=skf)
print("CV accuracy score: {:.2f}%".format(results_knn2.mean()*100))

CV accuracy score: -294.46%


In [77]:
knn2.score(X_test, Y_test)

0.2838904128703639

In [73]:
rg1 = Ridge(alpha=1)
rg1.fit(X_train, y_train)
results_rg1 = cross_val_score(rg1, X_train, y_train, cv=skf)
print("CV accuracy score: {:.2f}%".format(results_rg1.mean()*100))

CV accuracy score: -94.74%


In [74]:
rg1.score(X_test, Y_test)

0.7518096225409064

In [71]:
rg2 = Ridge(alpha=100.1)
rg2.fit(X_train, y_train)
results_rg2 = cross_val_score(rg2, X_train, y_train, cv=skf)
print("CV accuracy score: {:.2f}%".format(results_rg2.mean()*100))

CV accuracy score: -433.79%


In [72]:
rg2.score(X_test, Y_test)

0.7435448342601186

In [68]:
rg3 = Ridge(alpha=0.001)
rg3.fit(X_train, y_train)
results_rg3 = cross_val_score(rg3, X_train, y_train, cv=skf)
print("CV accuracy score: {:.2f}%".format(results_rg3.mean()*100))

CV accuracy score: -96.40%


In [70]:
rg3.score(X_test, Y_test)

0.7300119163198853

In [80]:
clf_lr = LogisticRegression(penalty='l1', C=1)
clf_lr.fit(X_train, y_train)
results_clf_lr = cross_val_score(clf_lr, X_train, y_train)
print("CV accuracy score: {:.2f}%".format(results_clf_lr.mean()*100))

CV accuracy score: 0.98%


In [81]:
clf_lr.score(X_test, Y_test)

0.00684931506849315

In [ ]:
clf_lr1 = LogisticRegression( C=100)
clf_lr1.fit(X_train, y_train)
results_clf_lr1 = cross_val_score(clf_lr1, X_train, y_train, cv=skf)
print("CV accuracy score: {:.2f}%".format(results_clf_lr1.mean()*100))

In [ ]:
clf_lr3 = LogisticRegression(penalty='l1', C=0.001)
clf_lr3.fit(X_train, y_train)
results_clf_lr3 = cross_val_score(clf_lr3, X_train, y_train, cv=skf)
print("CV accuracy score: {:.2f}%".format(results_clf_lr3.mean()*100))

In [ ]:
from sklearn.metrics import mean_squared_error
from math import sqrt

rms = sqrt(mean_squared_error(y_actual, y_predicted))

In [60]:
from sklearn.linear_model import LinearRegression

In [61]:
regression = LinearRegression()

In [62]:
regression.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [63]:
y_pred = regression.predict(X_test)

In [65]:
regression.score(X_test, Y_test)

0.7299864946381466

In [122]:
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_predict

class DjStacking(BaseEstimator, ClassifierMixin):  


    def __init__(self, models, ens_model):
        self.models = models
        self.ens_model = ens_model
        self.n = len(models)
        self.valid = None
        
    def fit(self, X, y=None, p=0.25, cv=3, err=0.001, random_state=None):
        if (p > 0):
            train, valid, y_train, y_valid = train_test_split(X_train, y, test_size=p, random_state=random_state)
            
            self.valid = np.zeros((valid.shape[0], self.n))
            for t, clf in enumerate(self.models):
                clf.fit(train, y_train)
                self.valid[:, t] = clf.predict(valid)
                
            self.ens_model.fit(self.valid, y_valid)
            
        return self
    


    def predict(self, X, y=None):
        """
        Работа стэкинга
        """
        X_meta = np.zeros((X.shape[0], self.n))
        
        for t, clf in enumerate(self.models):
            X_meta[:, t] = clf.predict(X)
        
        a = self.ens_model.predict(X_meta)
        
        return (a)

In [128]:
models = [regression, rg1 ,rfr] # , rf3
ens_model = Ridge()
s1 = DjStacking(models, ens_model)
s1.fit(X_train, y_train)

DjStacking(ens_model=Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001),
      models=[LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False), Ridge(alpha=1, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001), RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=100,
           max_...stimators=10, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)])

In [129]:
y_pred2 = s1.predict(X_test, Y_test)

In [130]:
rms = sqrt(mean_squared_error(Y_test, y_pred2))
rms

37972.38820686792